# m2.1t.jl

Load Julia packages (libraries) needed

In [1]:
using TuringModels
using Optim, Turing, Flux.Tracker
gr(size=(600,300));

In [2]:
Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false)

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/yF1Mq/src/Turing.jl:24


false

### snippet 2.8t

Define the data

In [3]:
k = 6; n = 9;

Define the model

In [4]:
@model globe_toss(n, k) = begin
  theta ~ Beta(1, 1) # prior
  k ~ Binomial(n, theta) # model
  return k, theta
end;

Compute the "maximum_a_posteriori" value

Set search bounds

In [5]:
lb = [0.0]; ub = [1.0];

Create (compile) the model

In [6]:
model = globe_toss(n, k);

Compute the maximum_a_posteriori

In [7]:
maximum_a_posteriori(model, lb, ub)

Results of Optimization Algorithm
 * Algorithm: Fminbox with L-BFGS
 * Starting Point: [0.026695322252818118]
 * Minimizer: [0.6666666664320972]
 * Minimum: 1.297811e+00
 * Iterations: 3
 * Convergence: true
   * |x - x'| ≤ 0.0e+00: false 
     |x - x'| = 2.13e-07 
   * |f(x) - f(x')| ≤ 0.0e+00 |f(x)|: false
     |f(x) - f(x')| = 7.09e-13 |f(x)|
   * |g(x)| ≤ 1.0e-08: true 
     |g(x)| = 8.76e-09 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 52
 * Gradient Calls: 52

Use Turing mcmc

In [8]:
chn = sample(model, Turing.NUTS(2000, 1000, 0.65));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/yF1Mq/src/inference/support/hmc_core.jl:245
┌ Info: [Turing] found initial ϵ: 1.2000000000000002
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/yF1Mq/src/inference/support/hmc_core.jl:237
┌ Info:  Adapted ϵ = 1.7914655084659348, std = [1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/yF1Mq/src/inference/adapt/adapt.jl:90
[NUTS] Finished with
  Running time        = 0.7821556540000012;
  #lf / sample        = 0.0;
  #evals / sample     = 5.7905;
  pre-cond. metric    = [1.0].


Look at the proper draws (in corrected chn2)

In [9]:
chn2 = chn[1001:2000, :, :]
describe(chn2)

Log evidence      = 0.0
Iterations        = 1001:2000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = theta

Empirical Posterior Estimates
───────────────────────────────────────────
parameters
       Mean    SD   Naive SE  MCSE    ESS  
theta 0.5848 0.1865   0.0059 0.0408 20.8818

Quantiles
───────────────────────────────────────────
parameters
       2.5%  25.0%  50.0%  75.0%  97.5%
theta 0.2759 0.455 0.6415 0.7224 0.9246



Show the hpd region

In [10]:
MCMCChains.hpd(chn2[:theta], alpha=0.055)

Log evidence      = 0.0
Iterations        = 1001:2000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = theta

parameters
      94.5% Lower 94.5% Upper
theta  0.27594955  0.84239432



End of `02/m2.1t.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*